# Method

### Import support packages

In [1]:
import os
import kagglehub
from DataGeneration import (
    get_season_mvp,
    collect_player_stats,
    normalize_player_stats,
    load_combined_mvp_2001_2023,
    fetch_mvp_stats_by_year,
    check_missing_players,
    print_missing_players,
    get_player_season,
    generate_3_wide_window,
)

from DataProcessing import (
    get_iterable_window_data,
)


### make directories
player_data_path = 'Raw_player_data_sets'
window_player_path = 'windowed_data_sets'

os.makedirs(player_data_path, exist_ok=True)
os.makedirs(window_player_path, exist_ok=True)


### Fetch and save dataset

Yearly MVP results

In [2]:
mvp_data = load_combined_mvp_2001_2023()
players_by_year = mvp_data.groupby('year')['Player'].apply(list).to_dict()


fetch_mvp_stats_by_year(mvp_data, False, player_data_path)

tempdic = check_missing_players(players_by_year,player_data_path)

2001 :  ['Allen Iverson', 'Tim Duncan', "Shaquille O'Neal", 'Chris Webber', 'Kevin Garnett', 'Tracy McGrady', 'Karl Malone', 'Jason Kidd', 'Kobe Bryant', 'David Robinson', 'Ray Allen', 'Vince Carter', 'Paul Pierce', 'Jerry Stackhouse', 'Michael Finley', 'Anthony Mason', 'John Stockton']
Data for year 2001 already exists at Raw_player_data_sets/mvp_stats_2001.csv. Checking formatting...
Column names reformatted for year 2001 if necessary. Skipping fetching...
2002 :  ['Tim Duncan', 'Jason Kidd', "Shaquille O'Neal", 'Tracy McGrady', 'Kobe Bryant', 'Gary Payton', 'Chris Webber', 'Dirk Nowitzki', 'Allen Iverson', 'Ben Wallace', 'Paul Pierce', 'Kevin Garnett', 'Michael Jordan', 'Steve Nash', 'Jerry Stackhouse', 'Mike Bibby', 'Elton Brand', 'Peja Stojaković']
Data for year 2002 already exists at Raw_player_data_sets/mvp_stats_2002.csv. Checking formatting...
Column names reformatted for year 2002 if necessary. Skipping fetching...
2003 :  ['Tim Duncan', 'Kevin Garnett', 'Kobe Bryant', 'Tracy

### Missing player entries:

In [3]:
print_missing_players(tempdic)

Year 2001: 1 missing player(s)
Missing Players:
 - Shaquille O'Neal
----------------------------------------
Year 2002: 1 missing player(s)
Missing Players:
 - Shaquille O'Neal
----------------------------------------
Year 2003: 1 missing player(s)
Missing Players:
 - Shaquille O'Neal
----------------------------------------
Year 2004: 2 missing player(s)
Missing Players:
 - Jermaine O'Neal
 - Shaquille O'Neal
----------------------------------------
Year 2005: 2 missing player(s)
Missing Players:
 - Shaquille O'Neal
 - P.J. Brown
----------------------------------------
Year 2007: 1 missing player(s)
Missing Players:
 - Shaquille O'Neal
----------------------------------------


### generate windows and new data sets

In [4]:
windowed_years = generate_3_wide_window(players_by_year)

get_iterable_window_data(players_by_year=players_by_year, data_path=player_data_path)



<generator object get_iterable_window_data at 0x148fadb90>

In [5]:
# Define the player data path and players_by_year dictionary for testing purposes
player_data_path = 'Raw_player_data_sets'
players_by_year = {
    2001: ['Allen Iverson', 'Tim Duncan', "Shaquille O'Neal"],
    2002: ['Tim Duncan', 'Jason Kidd', "Shaquille O'Neal"],
    2003: ['Tim Duncan', 'Kevin Garnett', 'Kobe Bryant']
}

# Get the iterator for window data
window_data_iterator = get_iterable_window_data(players_by_year, player_data_path)

# Save the combined sections from the first iteration
first_iteration = next(window_data_iterator)

# Save the combined sections as CSV files with meaningful names
first_iteration['combined_start_section'].to_csv('combined_start_section_window_2001_2003.csv', index=False)
first_iteration['combined_middle_section'].to_csv('combined_middle_section_window_2001_2003.csv', index=False)
first_iteration['combined_finish_section'].to_csv('combined_finish_section_window_2001_2003.csv', index=False)


Processing window with years: [2001, 2002, 2003]
